## Dash a plotly

# Plotly
- plotly.js 
- JSON - JavaScript Object Notation

- instalace
```
pip3 install dash plotly
```

In [ ]:
import numpy as np
import plotly.express as px

x = np.linspace(0, 2*np.pi, 1000)
y = np.sin(x)

fig = px.line(x = x, y = y, labels = {"x" : "xlabel", "y" : "ylabel"})
fig.show()
print(fig)

In [ ]:
import pandas as pd

df = pd.read_csv("nakazeni-vyleceni-umrti-testy.csv")

df.set_index(['datum'], drop = True, inplace = True)
df.index = pd.to_datetime(df.index)

new_names = ["inf", "cur", "dead", "tests", "ag", "d_inf", "d_cur", "d_dead", "d_tests", "d_ag"]
df.columns = new_names

df.head()

In [ ]:
import plotly.express as px

fig = px.line(df,
              y = ["d_inf", "d_dead"],
              title = "Vyvoj epidemie covid19",
              labels = {"datum" : "Datum", "d_inf" : "Pocet", "value": "Pocet"},
              template = "simple_white"
             )
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
d_dead = go.Scatter(x = df.index, y = df["d_dead"], name = "Denni prirustek zemrelych")
d_inf = go.Scatter(x = df.index, y = df["d_inf"], name = "Denni prirustek infikovanych")
fig.add_trace(d_dead)
fig.add_trace(d_inf)
fig.update_layout(template = "simple_white", title = "covid")
fig.show()

# Dash

- lze i v jupyteru
```
pip3 install jupyter-dash
```
- html - hypertext markup language
    - <h1>Nadpis prvni urovne</h1>
    - <div>oddeleny blok na strance </div>

In [ ]:
from jupyter_dash import JupyterDash
from dash import html
# import dash.dash_html_components

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Nazdar")
])

# app.run_server(mode = "inline")
app.run_server()

In [ ]:
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
# import dash.dash_core_components as dcc - deprecated


pozdrav = {
    "cs" : "Ahoj",
    "de" : "Hallo",
    "en" : "Hello"
}

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Ahoj", id = "nadpis"),
    dcc.RadioItems(id = "jazyk",
        options = [
            {"label" : "Česky", "value" : "cs"},
            {"label" : "Deutsch", "value" : "de"},
            {"label" : "English", "value" : "en"},
        ],
        value = "cs"
    )
])

@app.callback(
    Output("nadpis", "children"),
    Input("jazyk", "value")
    )
def update_hello(val):
    return pozdrav[val]

app.run_server(mode = "inline")

In [ ]:
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

import plotly.graph_objects as go

import pandas as pd

### DATA PREPROCESSING =====================================
df = pd.read_csv("nakazeni-vyleceni-umrti-testy.csv")

df.set_index(['datum'], drop = True, inplace = True)
df.index = pd.to_datetime(df.index)

new_names = ["inf", "cur", "dead", "tests", "ag", "d_inf", "d_cur", "d_dead", "d_tests", "d_ag"]
df.columns = new_names

pretty_names ={
    "inf" : "Nakažení celkem",
    "dead" : "Mrtví celkem",
    "d_inf" : "Přírůstek nakažených",
    "d_dead" : "Přírůstek mrtvých"
}

aggr = {"inf" : "sum", "dead" : "sum", "d_inf" : "mean",  "d_dead" : "mean"}
# =========================================================

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Covid"),
    dcc.Checklist(id="checklist-data",
                  options = [{'label' : pretty_names[x], "value" : x} for x in ["d_inf", "d_dead"]],
                  value = ["d_inf"]
                 ),
    dcc.Graph(id="graph"),
    dcc.Graph(id="graph2"),
])

@app.callback(
    Output("graph", "figure"),
    Input("checklist-data", "value")
)
def update_fig(keys, arg2):
    fig = go.Figure()
    for key in keys:
        trace = go.Scatter(x = df.index, y = df[key], name = pretty_names[key])
        fig.add_trace(trace)
    return fig

app.run_server(mode="inline")